In [51]:
import pandas as pd


In [67]:
excel_path = "../data/4_GEOMAR-relevant_COG_grantees_and_panel_members_2013-2024.xlsx"
sheet_name = "Panel members"
filter_call = "CoG 2024"

In [68]:
df = pd.read_excel(excel_path, sheet_name=sheet_name)
# strip the column names
df.columns = df.columns.str.strip().str.replace(r'\s+', ' ', regex=True)
display(df.head())

# filter
#df = df[df['Call'] == filter_call].reset_index(drop=True)

# Zeige alle Spaltennamen
print("Verfügbare Spalten:")
print(df.columns.tolist())



# Nur wenn die Spalten existieren
if 'First name' in df.columns and 'Last name' in df.columns:
    df['Name'] = df["First name"] + " " + df["Last name"].str.strip()
else:
    print("\n⚠️ Kann 'Name' nicht erstellen - Spalten fehlen!")

display(df.head())

,Last name,First name,Host institution (HI),Field,Panel,Function,Domain,Call
0,Agren,Jon,University of Uppsala,Ecology and evolution of plant populations; Ev...,LS8,Member,LS,CoG 2014
1,Agren,Jon,University of Uppsala,Ecology and evolution of plant populations; Ev...,LS8,Chair,LS,CoG 2016
2,Albarede,Francis,ENS Lyon,"geochemistry; Trace elements, radiogenic isoto...",PE10,Member,PE,CoG 2014
3,Albarede,Francis,ENS Lyon,"geochemistry; Trace elements, radiogenic isoto...",PE10,Member,PE,CoG 2016
4,Ameli,Nadia,UCL Institute for Sustainable Resources,"Climate Finance. climate finance, networks and...",SH7,Member,SH,CoG 2024


Verfügbare Spalten:
['Last name', 'First name', 'Host institution (HI)', 'Field', 'Panel', 'Function', 'Domain', 'Call']


,Last name,First name,Host institution (HI),Field,Panel,Function,Domain,Call,Name
0,Agren,Jon,University of Uppsala,Ecology and evolution of plant populations; Ev...,LS8,Member,LS,CoG 2014,Jon Agren
1,Agren,Jon,University of Uppsala,Ecology and evolution of plant populations; Ev...,LS8,Chair,LS,CoG 2016,Jon Agren
2,Albarede,Francis,ENS Lyon,"geochemistry; Trace elements, radiogenic isoto...",PE10,Member,PE,CoG 2014,Francis Albarede
3,Albarede,Francis,ENS Lyon,"geochemistry; Trace elements, radiogenic isoto...",PE10,Member,PE,CoG 2016,Francis Albarede
4,Ameli,Nadia,UCL Institute for Sustainable Resources,"Climate Finance. climate finance, networks and...",SH7,Member,SH,CoG 2024,Nadia Ameli


In [77]:
# show dataframe where Name is not null and other columnas contain Nan
df_member = df[df['Name'].notnull()].reset_index(drop=True)

empty_fields = df_member[df_member["Field"].isnull()]
display(empty_fields)

unique_members = empty_fields['Name'].unique()
print(f"\nAnzahl der einzigartigen Mitglieder ohne 'Field': {len(unique_members)}")



,Last name,First name,Host institution (HI),Field,Panel,Function,Domain,Call,Name
8,Anderluh,Gregor,NaN,NaN,LS9,Member,LS,CoG 2021,Gregor Anderluh
9,Anderluh,Gregor,NaN,NaN,LS9,Member,LS,CoG 2023,Gregor Anderluh
15,Arndt,Nicholas,NaN,NaN,PE10,Member,PE,CoG 2021,Nicholas Arndt
16,Arndt,Nicholas,NaN,NaN,PE10,Member,PE,CoG 2023,Nicholas Arndt
19,Bai,Yuling,NaN,NaN,LS9,Member,LS,CoG 2021,Yuling Bai
...,...,...,...,...,...,...,...,...,...
543,Virrantaus,Kirsi,NaN,NaN,SH7,Member,SH,CoG 2021,Kirsi Virrantaus
544,Virrantaus,Kirsi,NaN,NaN,SH7,Member,SH,CoG 2023,Kirsi Virrantaus
561,Wu,Xiaogang,NaN,NaN,SH7,Member,SH,CoG 2021,Xiaogang Wu
562,Xu,Chong-Yu,NaN,NaN,PE10,Member,PE,CoG 2021,Chong-Yu Xu



Anzahl der einzigartigen Mitglieder ohne 'Field': 68


In [78]:
unique_members

array(['Gregor Anderluh', 'Nicholas Arndt', 'Yuling Bai',
       'Michael Benton', 'Petra Boevink', 'Peter Bols',
       'Kim A. Boutilier', 'Donald E. Canfield', 'Jørgen Carling',
       'Paul Cloke', 'Marcus Collier', 'Pierre Phillippe Combes',
       'Pierre-Philippe Combes', 'Cláudia de Freitas', 'Ben Derudder',
       'Paul Ekins', 'Martin Funk', 'Toni Gabaldón', 'Andrey Ganopolski',
       'Alexandros Gasparatos', 'Ashleigh Griffin', 'Lars-Anders Hansson',
       'Marie E. Herberstein', 'Niels Høiby', 'Robert Holt',
       'Corinna Hoose', 'Dirk Jacobs', 'Adrian Jenkins', 'Nadja Kabisch',
       'Henrik Kaessmann', 'Inger Kappel Schmidt', 'Kristine Kern',
       'Phoebe Koundouri', 'Julie LaRoche', 'Iñigo Lasa Uzcudun',
       'Philippe Lemanceau', 'Edward Lemke', 'Regina Lindborg',
       'Rosaly Lopes', 'Alessandra Magistrato', 'Johanna Mappes',
       'Anders Meibom', 'Andrzej Mizgajski', 'Andrew Mondschein',
       'Michael Obersteiner', 'Spyros Pandis', 'Dario Papale',
     

In [31]:
import requests
from pprint import pprint


def fetch_openalex_id(firstname, lastname):
    base_url = f"https://api.openalex.org/authors?filter=display_name.search:{firstname}+{lastname}"

    response = requests.get(base_url)
    if response.status_code == 200:
        data = response.json()
        results = data.get('results', [])
        pprint(type(results[0]))
        pprint(results[0].keys())
        topics = results[0].get('topics', [])
        affiliations = results[0].get('affiliations', [])
        x_concepts = results[0].get('x_concepts', [])

        #pprint(topics)

        topics_fields_list = [topic['display_name'] for topic in topics]
        pprint(topics_fields_list)
        #pprint(affiliations)
        affiliation = affiliations[0]["institution"]["display_name"] if affiliations else None
        pprint(affiliation)
        #pprint(x_concepts)
        x_concepts_fields_list = [concept['display_name'] for concept in x_concepts]
        pprint(x_concepts_fields_list)
        
    return None



name = "Gregor Anderluh"
firstname, lastname = name.split(" ", 1)
openalex_id = fetch_openalex_id(firstname, lastname)
print(f"OpenAlex ID for {name}: {openalex_id}")

<class 'dict'>
dict_keys(['id', 'orcid', 'display_name', 'display_name_alternatives', 'relevance_score', 'works_count', 'cited_by_count', 'summary_stats', 'ids', 'affiliations', 'last_known_institutions', 'topics', 'topic_share', 'x_concepts', 'counts_by_year', 'works_api_url', 'updated_date', 'created_date'])
['Lipid Membrane Structure and Behavior',
 'Marine Invertebrate Physiology and Ecology',
 'RNA and protein synthesis mechanisms',
 'Mycotoxins in Agriculture and Food',
 'Erythrocyte Function and Pathophysiology',
 'Plant Pathogens and Fungal Diseases',
 'Cellular transport and secretion',
 'Marine Toxins and Detection Methods',
 'Bacterial Genetics and Biotechnology',
 'Listeria monocytogenes in Food Safety',
 'Bacteriophages and microbial interactions',
 'Nanopore and Nanochannel Transport Studies',
 'Toxin Mechanisms and Immunotoxins',
 'Venomous Animal Envenomation and Studies',
 'Glycosylation and Glycoproteins Research',
 'RNA Interference and Gene Delivery',
 'Hemoglobin s